In [13]:
import os
import re
import sys
import glob

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from Gaugi import load as gload

# add gaugi by hand
sys.path.append('../../.__python__/')
from kolmov import table_info, dump_all_train_history, dump_train_history

In [2]:
base_path   = '/volume/'
output_path = os.path.join(base_path, 'models_history_v1_nohad')
#task_name   = 'v1_task/user.mverissi.task.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*'
task_name   = 'v1_task_nohad/user.mverissi.task_nohad.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*'

task_full_name = os.path.join(base_path, task_name)

In [3]:
# check the paths
print(output_path)
print(task_full_name)

/volume/models_history_v1_nohad
/volume/v1_task_nohad/user.mverissi.task_nohad.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_*/*


In [4]:
# wanted information
tuned_info = {
    # validation
    'max_sp_val'     : [],
    'max_sp_pd_val'  : [],
    'max_sp_fa_val'  : [],
    'auc_val'        : [],
    # operation
    'max_sp_op'     : [],
    'max_sp_pd_op'  : [],
    'max_sp_fa_op'  : [],
    'auc_op'        : [],
}

In [5]:
# init the table_info
test = table_info(task_full_name, tuned_info, tag='v1_nohad')

There are 900 files for this task...


In [6]:
# fill table
test.fill_table()

Filling the table... 
End of fill step, a pandas DataFrame was created...


In [7]:
cv_df = test.get_pandas_table()
cv_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,total_sgn,total_bkg,max_sp_val,max_sp_pd_val,max_sp_fa_val,auc_val,max_sp_op,max_sp_pd_op,max_sp_fa_op,auc_op
0,v1_nohad,1,1,6,0,20,tunedDiscr.jobID_0032.pic.gz,6436,73903,0.951187,0.970497,0.067930,0.980453,0.940271,0.959447,0.078711,0.978734
1,v1_nohad,1,1,6,0,21,tunedDiscr.jobID_0032.pic.gz,6436,73903,0.951528,0.970497,0.067253,0.980277,0.939824,0.950901,0.071188,0.978293
2,v1_nohad,1,1,6,0,22,tunedDiscr.jobID_0032.pic.gz,6436,73903,0.950913,0.970497,0.068471,0.979746,0.939581,0.963021,0.083569,0.978272
3,v1_nohad,1,1,6,0,23,tunedDiscr.jobID_0032.pic.gz,6436,73903,0.951101,0.968944,0.066576,0.980681,0.940259,0.959602,0.078887,0.979097
4,v1_nohad,1,1,6,0,24,tunedDiscr.jobID_0032.pic.gz,6436,73903,0.951802,0.970497,0.066712,0.980147,0.940031,0.960690,0.080403,0.978634


In [8]:
# filter the DataFrame in order to get only 1 init.
# ordened by max_sp_val
cv_df = test.filter_inits('max_sp_val')

In [9]:
# only 1 init in each sort
len(cv_df)

1650

In [10]:
# save the filterd DataFrame
test.dump_table(cv_df, base_path, 'cv_table_v1_nohad')

In [11]:
cv_df.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,total_sgn,total_bkg,max_sp_val,max_sp_pd_val,max_sp_fa_val,auc_val,max_sp_op,max_sp_pd_op,max_sp_fa_op,auc_op
50158,v1_nohad,0,0,0,0,8,tunedDiscr.jobID_0000.pic.gz,28455,205005,0.941784,0.968377,0.084439,0.978992,0.938199,0.956247,0.079676,0.978314
51673,v1_nohad,0,0,0,1,43,tunedDiscr.jobID_0004.pic.gz,28455,205005,0.938560,0.960295,0.082927,0.976318,0.937698,0.956001,0.080427,0.978230
50693,v1_nohad,0,0,0,2,23,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.934502,0.941321,0.072293,0.975314,0.937448,0.954630,0.079579,0.977537
50718,v1_nohad,0,0,0,3,28,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.937192,0.956781,0.082195,0.978030,0.937805,0.957301,0.081491,0.978082
50730,v1_nohad,0,0,0,4,20,tunedDiscr.jobID_0002.pic.gz,28455,205005,0.938182,0.957836,0.081268,0.976505,0.938043,0.954490,0.078262,0.978116


In [12]:
# save all the train evolution into json files
task_p = '/volume/v1_task_nohad/user.mverissi.task_nohad.data17_13TeV.AllPeriods.sgn.probes_lhmedium_EGAM2.bkg.VProbes_EGAM7.GRL_v97_et%i_eta%i.v1'
dump_all_train_history(cplot_monitoring()v_df, task_p, output_path)